In [1]:
import requests
from bs4 import BeautifulSoup
import ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ChatMessage
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_ollama.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
import numpy as np

instruct_llm = ChatOllama(model="llama3.2") | StrOutputParser()


In [2]:
summary_sys_msg = (
    "You are an expert in summarizing about a company."
)


response_prompt = ChatPromptTemplate.from_messages([("system", "{system}"), ("user", "{prompt} {webcontent}")])

web_chain = (
     RunnableAssign(dict(
        system = RunnableLambda(lambda x: summary_sys_msg))        
    ) | response_prompt | instruct_llm
) 

def info_gen(message, return_buffer=False):
    buffer = ""
    for token in web_chain.stream(message):
        buffer += token
        yield buffer if return_buffer else token


In [10]:
prompt = "tell me a summary of the company www.cincinnatidrywall.net and its major products"
for token in info_gen({"webcontent":"", "prompt":prompt}):
    print(token, end='')

**Company Summary:**

Cincinnati Drywall is a leading provider of drywall installation, repair, and finishing services in the Midwest region of the United States. The company was established to serve residential and commercial clients with high-quality drywall solutions.

**History:**
Established in 2006, Cincinnati Drywall has grown rapidly through word-of-mouth referrals, exceptional customer service, and a commitment to quality workmanship.

**Services and Products:**

1. **Drywall Installation**: Expert installation of new drywall for residential and commercial projects.
2. **Drywall Repair**: Fast and reliable repair services for damaged or torn drywall, including patching and finishing.
3. **Drywall Finishing**: Professional finishing services to prepare drywall surfaces for painting or other finishes.

**Major Products/Services:**

1. **Acoustic Drywall**: High-density drywall designed to reduce echo and improve soundproofing in homes and commercial spaces.
2. **Moisture-Resista

In [4]:
def crawl(url, max_depth=1):
    """Crawls a website and extracts links up to a specified depth."""
    content = "";
    html=""
    visited = set()
    queue = [(url, 0)]

    while queue:
        url, depth = queue.pop(0)
        if url in visited or depth > max_depth:
            continue

        visited.add(url)

        try:
            headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
        }

            response = requests.get(url, headers=headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text(separator='\n', strip=True)
            content = content + text

            print(f"Crawling: {url}")

            for link in soup.find_all('a', href=True):
                href = link['href']
                if href.startswith('http') and href.startswith(url):
                    queue.append((href, depth + 1))
        
        except requests.exceptions.RequestException as e:
            print(f"Error crawling {url}: {e}")
    return content, html

In [9]:
start_url = "https://www.chipotle.com"
content, html = crawl(start_url, max_depth=1)
print(content)

Crawling: https://www.chipotle.com
Mexican Food - Restaurant & Catering - Chipotle Mexican Grill
ENABLE ACCESSIBILITY
MENU
CATERING
REWARDS
OUR VALUES
NUTRITION
Tender, hand-chopped, seasoned,
and finished with our unique brisket
sauce made with smoky chili peppers.
Tender, hand-chopped, seasoned,
and finished with our unique brisket
sauce made with smoky chili peppers.
ORDER NOW
ORDER NOW
JOIN CHIPOTLE REWARDS. UNLOCK FREE CHIPOTLE.
CREATE AN ACCOUNT
OR
SIGN IN
SMOKED
BRISKET IS
BACK FOR A
LIMITED TIME
Tender, hand-chopped, seasoned with Mexican spices, then finished in our unique brisket sauce made with smoky chili peppers.
SMOKED BRISKET IS
BACK FOR A LIMITED
TIME
Tender, hand-chopped, seasoned with Mexican spices, then finished in our unique brisket sauce made with smoky chili peppers.
ORDER NOW
ORDER NOW
INTRODUCING
YOUR NEW
PERFECT
PAIRING
Discover refreshing new flavors that complement our real ingredients in your go-to order.
INTRODUCING YOUR
NEW PERFECT
PAIRING
Discover refres

In [6]:
prompt = "tell me a summary of chipotle company and its major products  "
for token in info_gen({"webcontent":"", "prompt":prompt}):
    print(token, end='')

Here's a summary of Chipotle Mexican Grill, Inc. and its major products:

**Company Overview**

Chipotle Mexican Grill, Inc. is a fast-casual restaurant chain that specializes in serving Mexican-inspired food made with responsibly sourced ingredients. The company was founded in 1993 by Steve Ells in Denver, Colorado, and has since grown to become one of the largest fast-food chains in the United States.

**Major Products**

Chipotle's menu offerings include:

1. **Burritos**: Large flour tortillas filled with beans, rice, meat (chicken, steak, carnitas, or barbacoa), cheese, salsa, and guacamole.
2. **Bowls**: Similar to burritos, but served in a bowl instead of wrapped in a tortilla.
3. **Tacos**: Soft corn tortillas filled with meat, beans, cheese, salsa, and guacamole.
4. **Salads**: A variety of greens topped with grilled chicken, steak, or carnitas, along with beans, cheese, and salsa.
5. **Chile Verde**: A dish made with slow-cooked green chiles, chicken or beef, rice, beans, and

In [7]:
prompt = "tell me a summary of the company and its major products based on the information given below. Strictly use the information given here. Do not use other information in the summary "
for token in info_gen({"webcontent":content, "prompt":prompt}):
    print(token, end='')

Here is a summary of the company and its major products based on the information provided:

Chipotle Mexican Grill is a restaurant and catering service that serves Mexican food. Their menu includes burritos, chips & salsa, and other items. 

Notable products include their Smoked Brisket, which is made with hand-chopped meat seasoned with Mexican spices and finished in their unique brisket sauce made with smoky chili peppers. They also offer catering services for groups of 6 to 200 people, starting at $8.25 per person.

They have introduced a new "Perfect Pairing" concept that complements their real ingredients in their go-to order.

In [8]:
prompt = "tell me a summary of www.chipotle.com"
for token in info_gen({"webcontent":"", "prompt":prompt}):
    print(token, end='')

Here's a summary of Chipotle Mexican Grill (www.chipotle.com):

**Company Overview:**

Chipotle Mexican Grill is a fast-casual restaurant chain that specializes in Mexican-inspired cuisine. The company was founded in 1993 by Steve Ells and is headquartered in Denver, Colorado.

**Mission and Values:**

Chipotle's mission is to "Deliver Food with Integrity" by serving high-quality, sustainably raised ingredients, responsibly sourced from farmers who share our values of sustainability, fairness, and community. The company's core values include:

* Respect for the food you put in your body
* Care for the people who make it happen
* Responsibility to the environment

**Menu:**

Chipotle's menu features a range of burritos, bowls, tacos, salads, and vegetarian options made with responsibly sourced ingredients such as chicken, steak, carnitas (pork), chorizo, and vegetarian options like beans, cheese, and roasted vegetables.

**Sustainability Efforts:**

Chipotle has implemented various sust